In [1]:
import pandas as pd
import numpy as np

In [2]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','')
    ds[col] = np.where(ds[col]=='',np.nan,ds[col])
    ds[col] = ds[col].astype('float32')
    return ds[col].astype('float32')

In [3]:
clicks_df = pd.read_csv('data/clicks.csv', low_memory=False,dtype={'advertiser_id':'int32','action_id':'float32','source_id':'int32','country_code':'category',\
                                                 'latitude':'float32','longitude':'float32','carrier_id':'float32','os_minor':'category',\
                                                  'os_major':'category','specs_brand':'category','timeToClick':'float32','ref_type':'category'})

clicks_df['touchX'] = fix_str_float(clicks_df,'touchX')
clicks_df['touchY'] = fix_str_float(clicks_df,'touchY')
clicks_df['created'] = pd.to_datetime(clicks_df['created'])
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'category',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
events_df.drop(columns=['device_countrycode','session_user_agent','ip_address','device_language'], inplace=True)
installs_df = pd.read_csv('data/installs.csv', low_memory=False, dtype={'ref_type':'category','application_id':'category',\
                                                      'device_brand':'category','kind':'category'})
installs_df['kind'] = installs_df['kind'].str.lower()
installs_df.drop(columns=['session_user_agent','ip_address','device_language'], inplace=True)
installs_df['wifi'].astype('bool', inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
installs_df.drop(['device_countrycode'], axis=1, inplace=True)
auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category',\
                                                                        'ref_type_id':'category','source_id':'category'})

auctions_df['date'] = pd.to_datetime(auctions_df['date'])
print('setup done')

/Users/sergiolch/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/sergiolch/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


setup done


In [4]:
installs_df.head(5)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,user_agent,event_uuid,kind,wifi,trans_id
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,NaN,3.739127e+17,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,NaN,7.805539e+18,NaN,NaN,NaN,NaN,NaN
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,NaN,8.355496e+18,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,NaN,2.355772e+18,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,NaN,6.156971e+18,NaN,NaN,NaN,NaN,NaN


In [5]:
events_df['connection_type'].describe()

count       1809296
unique            4
top       Cable/DSL
freq        1291512
Name: connection_type, dtype: object

In [6]:
events_df['connection_type'].value_counts()

Cable/DSL    1291512
Cellular      517204
Corporate        527
Dialup            53
Name: connection_type, dtype: int64

In [7]:
events_df['connection_type'].isnull().sum()


5935285

In [70]:
auctions_df = auctions_df.sort_values(by=['device_id','date'])
auctions_df['date_dif'] = auctions_df['date'].shift(periods=-1) - auctions_df['date']
auctions_df['in_seconds'] = np.nan
last_row = False
last_index = False
for index, row in auctions_df.iterrows():
    if not(isinstance(last_row, bool)):
        if row['device_id']!=last_row['device_id']:
            auctions_df.at[last_index,'date_dif'] = np.nan
    auctions_df.at[index,'in_seconds'] = row['date_dif'].total_seconds()
    last_row = row
    last_index = index
auctions_df['in_seconds'] = np.where(auctions_df['date_dif'].isnull(), np.nan, auctions_df['in_seconds'])

In [74]:
auctions_df.head(50)

,date,device_id,ref_type_id,source_id,date_dif,in_seconds
36601349,2019-04-23 15:00:57.222979,40621409780134,1,0,NaT,NaN
30900602,2019-04-19 19:40:28.465866,41863526108385,1,8,07:11:58.427014,25918.427014
14875535,2019-04-20 02:52:26.892880,41863526108385,1,3,00:06:35.616350,395.616350
32062481,2019-04-20 02:59:02.509230,41863526108385,1,3,00:06:59.166558,419.166558
45613542,2019-04-20 03:06:01.675788,41863526108385,1,3,00:02:55.712372,175.712372
42215232,2019-04-20 03:08:57.388160,41863526108385,1,3,00:02:29.075743,149.075743
22374394,2019-04-20 03:11:26.463903,41863526108385,1,3,00:00:14.547768,14.547768
12690007,2019-04-20 03:11:41.011671,41863526108385,1,3,00:00:45.669391,45.669391
42215538,2019-04-20 03:12:26.681062,41863526108385,1,5,00:00:30.801420,30.801420
12690107,2019-04-20 03:12:57.482482,41863526108385,1,3,00:01:58.661626,118.661626


In [75]:
auctions_df.to_csv('data/auctions_seconds.csv')

In [15]:
datos = pd.merge(auctions_df, installs_df, left_on='device_id', right_on='ref_hash', how='left')

In [9]:
installs_df.shape

(481511, 14)

In [10]:
auctions_df.shape

(47409528, 4)

In [17]:
datos.shape

(52602582, 18)

In [18]:
datos.head(5).sort_values(by=['date','device_id'])

,date,device_id,ref_type_id,source_id,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_brand,device_model,user_agent,event_uuid,kind,wifi,trans_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0,2019-04-21 17:52:11.181,302,1891515180541284343,2.564673e+18,NaN,False,False,2.987569314309514e+18,4.047342e+18,Dalvik/2.1.0 (Linux; U; Android 7.0; Redmi Not...,NaN,NaN,True,NaN
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0,2019-04-22 00:20:36.406,309,1494519392962156891,4.441122e+18,NaN,False,True,NaN,7.538551e+18,NaN,7eb05e7f-d62d-419f-b732-3d6b6765ee57,app_open,NaN,NaN
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0,2019-04-21 14:14:23.301,210,1891515180541284343,6.416039e+18,NaN,False,False,NaN,5.633254e+18,Dalvik/2.1.0 (Linux; U; Android 7.0; SM-J701M ...,NaN,NaN,NaN,NaN
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
test1 = auctions_df[['date', 'device_id']]

In [21]:
test1 = test1.sort_values(by=['device_id','date'])

In [22]:
test1 = test1.head(1000)

In [24]:
test1.shape

(1000, 2)

In [29]:
test1.head(50)

,date,device_id,date_dif
36601349,2019-04-23 15:00:57.222979,40621409780134,NaT
30900602,2019-04-19 19:40:28.465866,41863526108385,-4 days +04:39:31.242887
14875535,2019-04-20 02:52:26.892880,41863526108385,0 days 07:11:58.427014
32062481,2019-04-20 02:59:02.509230,41863526108385,0 days 00:06:35.616350
45613542,2019-04-20 03:06:01.675788,41863526108385,0 days 00:06:59.166558
42215232,2019-04-20 03:08:57.388160,41863526108385,0 days 00:02:55.712372
22374394,2019-04-20 03:11:26.463903,41863526108385,0 days 00:02:29.075743
12690007,2019-04-20 03:11:41.011671,41863526108385,0 days 00:00:14.547768
42215538,2019-04-20 03:12:26.681062,41863526108385,0 days 00:00:45.669391
12690107,2019-04-20 03:12:57.482482,41863526108385,0 days 00:00:30.801420


In [28]:
test1['date_dif'] = test1['date'].shift(periods=-1) - test1['date']
test1['in_seconds'] = np.nan

In [68]:
last_row = False
last_index = False
for index, row in test1.iterrows():
    if not(isinstance(last_row, bool)):
        if row['device_id']!=last_row['device_id']:
            test1.at[last_index,'date_dif'] = np.nan
    test1.at[index,'in_seconds'] = row['date_dif'].total_seconds()
    last_row = row
    last_index = index

In [58]:
test1.head(50)

,date,device_id,date_dif,in_seconds
36601349,2019-04-23 15:00:57.222979,40621409780134,NaT,NaN
30900602,2019-04-19 19:40:28.465866,41863526108385,NaT,NaN
14875535,2019-04-20 02:52:26.892880,41863526108385,07:11:58.427014,25918.427014
32062481,2019-04-20 02:59:02.509230,41863526108385,00:06:35.616350,395.616350
45613542,2019-04-20 03:06:01.675788,41863526108385,00:06:59.166558,419.166558
42215232,2019-04-20 03:08:57.388160,41863526108385,00:02:55.712372,175.712372
22374394,2019-04-20 03:11:26.463903,41863526108385,00:02:29.075743,149.075743
12690007,2019-04-20 03:11:41.011671,41863526108385,00:00:14.547768,14.547768
42215538,2019-04-20 03:12:26.681062,41863526108385,00:00:45.669391,45.669391
12690107,2019-04-20 03:12:57.482482,41863526108385,00:00:30.801420,30.801420


In [61]:
test1['date_dif'] = test1['date'].shift(periods=-1) - test1['date']

In [69]:
test1.head(50)

,date,device_id,date_dif,in_seconds
36601349,2019-04-23 15:00:57.222979,4.062141e+13,NaT,NaN
30900602,2019-04-19 19:40:28.465866,4.186353e+13,07:11:58.427014,25918.427014
14875535,2019-04-20 02:52:26.892880,4.186353e+13,00:06:35.616350,395.616350
32062481,2019-04-20 02:59:02.509230,4.186353e+13,00:06:59.166558,419.166558
45613542,2019-04-20 03:06:01.675788,4.186353e+13,00:02:55.712372,175.712372
42215232,2019-04-20 03:08:57.388160,4.186353e+13,00:02:29.075743,149.075743
22374394,2019-04-20 03:11:26.463903,4.186353e+13,00:00:14.547768,14.547768
12690007,2019-04-20 03:11:41.011671,4.186353e+13,00:00:45.669391,45.669391
42215538,2019-04-20 03:12:26.681062,4.186353e+13,00:00:30.801420,30.801420
12690107,2019-04-20 03:12:57.482482,4.186353e+13,00:01:58.661626,118.661626
